In [110]:
import pandas as pd
import os
import ast
from dotenv import load_dotenv
from pymongo import MongoClient

In [111]:
load_dotenv()

True

In [112]:
client = MongoClient(os.getenv("url"))

In [277]:
df = pd.read_csv("eurocup_2020_results.csv")

# Objetivo

Voy a organizar los datos de tal forma que pueda tener un json que cada objeto sea un pais con sus estadisticas, mientras que creare otro json para los jugadores. Finalmente, quizas, cree otro json que los objetos sean las fases de grupo.

- Datos por pais:
    - goles totales a favor/ en contra
    - goles medios por partido a favor/ encontra
    - victorias, empates, derrotas
    - posesion media por partido
    - disparos medios por partido totales
    - diparos medios por partido on target
    - eficacia media: sumatorio (total de disparos on target / total de disparos) / total partidos
    - disparos medios recibidos por partido
    - media de duelos ganados por partido
    - media de tarjetas amarillas / rojas
    - media de cambios por partido
    - rondas de clasificacion

- Datos por jugador:
    - partidos totales jugados
    - titular / suplente
    - minutos totales jugados
    - tarjetas
    - goles
    - numero, nombre
    - media de en que minuto le han cambiado

# 1. Anaisis previo, nulls y nan

In [5]:
df

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,11.07.2021,True,3,2,Italy,England,1,1,66%,34%,19,6,6,2,53%,47%,"[{'event_team': 'away', 'event_time': "" 2' "", ...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
1,Semi-finals,07.07.2021,False,False,False,England,Denmark,2,1,59%,41%,20,6,10,3,50%,50%,"[{'event_team': 'away', 'event_time': "" 30' "",...","[{'Player_Name': 'Kane', 'Player_Number': '9'}...","[{'Player_Name': 'Krogh Damsgaard', 'Player_Nu..."
2,Semi-finals,06.07.2021,True,4,2,Italy,Spain,1,1,29%,71%,7,16,4,5,49%,51%,"[{'event_team': 'away', 'event_time': "" 51' "",...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Torres', 'Player_Number': '1..."
3,Quarter-finals,03.07.2021,False,False,False,Ukraine,England,0,4,48%,52%,7,10,2,6,42%,59%,"[{'event_team': 'away', 'event_time': "" 4' "", ...","[{'Player_Name': 'Yaremchuk', 'Player_Number':...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
4,Quarter-finals,03.07.2021,False,False,False,Czech Republic,Denmark,1,2,56%,44%,16,11,5,7,48%,52%,"[{'event_team': 'away', 'event_time': "" 5' "", ...","[{'Player_Name': 'Schick', 'Player_Number': '1...","[{'Player_Name': 'Dolberg', 'Player_Number': '..."
5,Quarter-finals,02.07.2021,False,False,False,Belgium,Italy,1,2,46%,54%,10,14,4,3,58%,42%,"[{'event_team': 'away', 'event_time': "" 13' "",...","[{'Player_Name': 'Lukaku', 'Player_Number': '9...","[{'Player_Name': 'Insigne', 'Player_Number': '..."
6,Quarter-finals,02.07.2021,True,1,3,Switzerland,Spain,1,1,27%,73%,8,28,2,10,58%,42%,"[{'event_team': 'away', 'event_time': "" 8' "", ...","[{'Player_Name': 'Seferovic', 'Player_Number':...","[{'Player_Name': 'Sarabia', 'Player_Number': '..."
7,Round of 16,29.06.2021,False,False,False,Sweden,Ukraine,1,2,45%,55%,13,15,3,4,48%,52%,"[{'event_team': 'away', 'event_time': "" 27' "",...","[{'Player_Name': 'Isak', 'Player_Number': '11'...","[{'Player_Name': 'Yaremchuk', 'Player_Number':..."
8,Round of 16,29.06.2021,False,False,False,England,Germany,2,0,45%,55%,5,9,4,3,55%,45%,"[{'event_team': 'home', 'event_time': "" 8' "", ...","[{'Player_Name': 'Sterling', 'Player_Number': ...","[{'Player_Name': 'Werner', 'Player_Number': '1..."
9,Round of 16,28.06.2021,True,4,5,France,Switzerland,3,3,55%,45%,26,12,8,5,55%,45%,"[{'event_team': 'away', 'event_time': "" 15' "",...","[{'Player_Name': 'Mbappe', 'Player_Number': '1...","[{'Player_Name': 'Seferovic', 'Player_Number':..."


In [6]:
df.info()
#no hay nulos
#hay varios int que son str , habra que limpiarlos
#hay booleanos que son str, hay que limpiarlos
#las tres ultimas columnas contienen datos que tenemos que extraer y organizar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   stage                 51 non-null     object
 1   date                  51 non-null     object
 2   pens                  51 non-null     bool  
 3   pens_home_score       51 non-null     object
 4   pens_away_score       51 non-null     object
 5   team_name_home        51 non-null     object
 6   team_name_away        51 non-null     object
 7   team_home_score       51 non-null     int64 
 8   team_away_score       51 non-null     int64 
 9   possession_home       51 non-null     object
 10  possession_away       51 non-null     object
 11  total_shots_home      51 non-null     int64 
 12  total_shots_away      51 non-null     int64 
 13  shots_on_target_home  51 non-null     int64 
 14  shots_on_target_away  51 non-null     int64 
 15  duels_won_home        51 non-null     obje

In [25]:
ast.literal_eval(df["events_list"][0])

[{'event_team': 'away',
  'event_time': " 2' ",
  'event_type': 'Goal',
  'action_player_1': ' Luke Shaw ',
  'action_player_2': ' Kieran Trippier '},
 {'event_team': 'home',
  'event_time': " 47' ",
  'event_type': 'Yellow card',
  'action_player_1': ' Nicolo Barella '},
 {'event_team': 'home',
  'event_time': " 54' ",
  'event_type': 'Substitution',
  'action_player_1': ' Bryan Cristante ',
  'action_player_2': ' Nicolo Barella '},
 {'event_team': 'home',
  'event_time': " 55' ",
  'event_type': 'Yellow card',
  'action_player_1': ' Leonardo Bonucci '},
 {'event_team': 'home',
  'event_time': " 55' ",
  'event_type': 'Substitution',
  'action_player_1': ' Domenico Berardi ',
  'action_player_2': ' Ciro Immobile '},
 {'event_team': 'home',
  'event_time': " 67' ",
  'event_type': 'Goal',
  'action_player_1': ' Leonardo Bonucci '},
 {'event_team': 'away',
  'event_time': " 70' ",
  'event_type': 'Substitution',
  'action_player_1': ' Bukayo Saka ',
  'action_player_2': ' Kieran Trippie

In [21]:
for i in ast.literal_eval(df["events_list"][0]):
    print(i["event_type"])

Goal
Yellow card
Substitution
Yellow card
Substitution
Goal
Substitution
Substitution
Yellow card
Substitution
Yellow card
Substitution
Substitution
Substitution
Yellow card
Yellow card
Substitution
Substitution
Substitution
PK
PK
PK
PK
PK
PK
PK
PK
PK
PK


In [ ]:
ast.literal_eval(df["events_list"][0])

In [26]:
ast.literal_eval(df["lineup_home"][0])

[{'Player_Name': 'Insigne', 'Player_Number': '10'},
 {'Player_Name': 'Immobile', 'Player_Number': '17'},
 {'Player_Name': 'Chiesa', 'Player_Number': '14'},
 {'Player_Name': 'Verratti', 'Player_Number': '6'},
 {'Player_Name': 'Jorginho', 'Player_Number': '8'},
 {'Player_Name': 'Barella', 'Player_Number': '18'},
 {'Player_Name': 'Emerson', 'Player_Number': '13'},
 {'Player_Name': 'Chiellini', 'Player_Number': '3'},
 {'Player_Name': 'Bonucci', 'Player_Number': '19'},
 {'Player_Name': 'Di Lorenzo', 'Player_Number': '2'},
 {'Player_Name': 'Donnarumma', 'Player_Number': '21'}]

# 2. Limpieza y homogenizacion de los datos

### Primer vistazo

In [27]:
df

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,11.07.2021,True,3,2,Italy,England,1,1,66%,34%,19,6,6,2,53%,47%,"[{'event_team': 'away', 'event_time': "" 2' "", ...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
1,Semi-finals,07.07.2021,False,False,False,England,Denmark,2,1,59%,41%,20,6,10,3,50%,50%,"[{'event_team': 'away', 'event_time': "" 30' "",...","[{'Player_Name': 'Kane', 'Player_Number': '9'}...","[{'Player_Name': 'Krogh Damsgaard', 'Player_Nu..."
2,Semi-finals,06.07.2021,True,4,2,Italy,Spain,1,1,29%,71%,7,16,4,5,49%,51%,"[{'event_team': 'away', 'event_time': "" 51' "",...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Torres', 'Player_Number': '1..."
3,Quarter-finals,03.07.2021,False,False,False,Ukraine,England,0,4,48%,52%,7,10,2,6,42%,59%,"[{'event_team': 'away', 'event_time': "" 4' "", ...","[{'Player_Name': 'Yaremchuk', 'Player_Number':...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
4,Quarter-finals,03.07.2021,False,False,False,Czech Republic,Denmark,1,2,56%,44%,16,11,5,7,48%,52%,"[{'event_team': 'away', 'event_time': "" 5' "", ...","[{'Player_Name': 'Schick', 'Player_Number': '1...","[{'Player_Name': 'Dolberg', 'Player_Number': '..."
5,Quarter-finals,02.07.2021,False,False,False,Belgium,Italy,1,2,46%,54%,10,14,4,3,58%,42%,"[{'event_team': 'away', 'event_time': "" 13' "",...","[{'Player_Name': 'Lukaku', 'Player_Number': '9...","[{'Player_Name': 'Insigne', 'Player_Number': '..."
6,Quarter-finals,02.07.2021,True,1,3,Switzerland,Spain,1,1,27%,73%,8,28,2,10,58%,42%,"[{'event_team': 'away', 'event_time': "" 8' "", ...","[{'Player_Name': 'Seferovic', 'Player_Number':...","[{'Player_Name': 'Sarabia', 'Player_Number': '..."
7,Round of 16,29.06.2021,False,False,False,Sweden,Ukraine,1,2,45%,55%,13,15,3,4,48%,52%,"[{'event_team': 'away', 'event_time': "" 27' "",...","[{'Player_Name': 'Isak', 'Player_Number': '11'...","[{'Player_Name': 'Yaremchuk', 'Player_Number':..."
8,Round of 16,29.06.2021,False,False,False,England,Germany,2,0,45%,55%,5,9,4,3,55%,45%,"[{'event_team': 'home', 'event_time': "" 8' "", ...","[{'Player_Name': 'Sterling', 'Player_Number': ...","[{'Player_Name': 'Werner', 'Player_Number': '1..."
9,Round of 16,28.06.2021,True,4,5,France,Switzerland,3,3,55%,45%,26,12,8,5,55%,45%,"[{'event_team': 'away', 'event_time': "" 15' "",...","[{'Player_Name': 'Mbappe', 'Player_Number': '1...","[{'Player_Name': 'Seferovic', 'Player_Number':..."


In [67]:
#funcion para saber si hay mas de un tipo de objeto en una columna
def types(columna):
    sets = set()
    sets.update([type(i) for i in columna])
    return sets

In [78]:
#comprobamos los types de cada columna y ver si en alguna hay mas de uno
for i in df:
    print(f'{i}: {types(df[i])}')

stage: {<class 'str'>}
date: {<class 'str'>}
pens: {<class 'bool'>}
pens_home_score: {<class 'str'>}
pens_away_score: {<class 'str'>}
team_name_home: {<class 'str'>}
team_name_away: {<class 'str'>}
team_home_score: {<class 'int'>}
team_away_score: {<class 'int'>}
possession_home: {<class 'str'>}
possession_away: {<class 'str'>}
total_shots_home: {<class 'int'>}
total_shots_away: {<class 'int'>}
shots_on_target_home: {<class 'int'>}
shots_on_target_away: {<class 'int'>}
duels_won_home: {<class 'str'>}
duels_won_away: {<class 'str'>}
events_list: {<class 'str'>}
lineup_home: {<class 'str'>}
lineup_away: {<class 'str'>}


### Comprobamos los datos de cada columna

#### columna stage, date y pens

In [ ]:
#todas ok

#### columnas pens_home_score y pens_away_score

In [ ]:
#es str y necesito pasarlo a bool e ints

In [136]:
#creamos una funcion para cambiar los numeros a enteros y los F/T a bool.
def entero(valor):
    if valor == "False":
        return False
    if valor == "True":
        return True
    if valor.isalpha:
        return int(valor)

In [278]:
#aplicamos la funcion, a las columnas 
df["pens_home_score"] = df["pens_home_score"].apply(entero)
df["pens_away_score"] = df["pens_away_score"].apply(entero)

In [279]:
#compurebo que se han hecho los cambios
print(types(df["pens_home_score"]))
print(types(df["pens_home_score"]))

{<class 'int'>, <class 'bool'>}
{<class 'int'>, <class 'bool'>}


#### col team_name_home	team_name_away	

In [232]:
df["team_name_home"][0] #vemos que los nombres tienen espacios, vamos a eliminarlos

' Italy '

In [233]:
#funcion para eliminar los espacios tanto al principio como al final de un str
def no_spaces(data):
    return data.strip()

In [280]:
df["team_name_home"] = df["team_name_home"].apply(no_spaces)
df["team_name_away"] = df["team_name_away"].apply(no_spaces)

In [281]:
#comprobamos que se han eliminado los espacios
df["team_name_home"][0]

'Italy'

#### col team_home_score	team_away_score

In [ ]:
# colunnas ok, todos son enteros

#### col possession_home	possession_away

In [236]:
df["possession_home"][0]

'66%'

In [237]:
#hay que pasarlos a enteros y quitar el %

def per_int(value):
        return int(value.replace(value[value.find("%")], ""))

In [282]:
df["possession_home"] = df["possession_home"].apply(per_int)
df["possession_away"] = df["possession_away"].apply(per_int)

In [283]:
type(df["possession_home"][0])

numpy.int64

#### col total_shots_home	total_shots_away

In [246]:
#todo ok

#### col shots_on_target_home	shots_on_target_away

In [ ]:
#todo ok

#### col duels_won_home	duels_won_away

In [284]:
#igual que con las de posesion, aplicamos la mimsa formula
df["duels_won_home"][0]

'53%'

In [285]:
df["duels_won_home"] = df["duels_won_home"].apply(per_int)
df["duels_won_away"] = df["duels_won_away"].apply(per_int)

In [286]:
type(df["duels_won_home"][0])

numpy.int64

#### col events_list , lineup_home,  lineup_away

In [287]:
#con la funcion ast.literal_eval(), logramos obtener una lista de diccionarios mucho mas limpia y accesible
# ademas, de pasarlo de str a lista

In [288]:
def lst_ok(value):
    return ast.literal_eval(value)

In [289]:
df1 = df

In [290]:
df["events_list"] = df["events_list"].apply(lst_ok)
df["lineup_home"] = df["lineup_home"].apply(lst_ok)
df["lineup_away"] = df["lineup_away"].apply(lst_ok)

In [297]:
df["events_list"][0][0]

{'event_team': 'away',
 'event_time': " 2' ",
 'event_type': 'Goal',
 'action_player_1': ' Luke Shaw ',
 'action_player_2': ' Kieran Trippier '}

In [294]:
df["lineup_home"][0][0]

{'Player_Name': 'Insigne', 'Player_Number': '10'}

In [298]:
df["lineup_away"][0][0]

{'Player_Name': 'Kane', 'Player_Number': '9'}

## Crear dataframe de los paises

In [299]:
df_clean = df

In [363]:
df_clean.head()

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,11.07.2021,True,3,2,Italy,England,1,1,66,34,19,6,6,2,53,47,"[{'event_team': 'away', 'event_time': ' 2' ', ...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
1,Semi-finals,07.07.2021,False,False,False,England,Denmark,2,1,59,41,20,6,10,3,50,50,"[{'event_team': 'away', 'event_time': ' 30' ',...","[{'Player_Name': 'Kane', 'Player_Number': '9'}...","[{'Player_Name': 'Krogh Damsgaard', 'Player_Nu..."
2,Semi-finals,06.07.2021,True,4,2,Italy,Spain,1,1,29,71,7,16,4,5,49,51,"[{'event_team': 'away', 'event_time': ' 51' ',...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Torres', 'Player_Number': '1..."
3,Quarter-finals,03.07.2021,False,False,False,Ukraine,England,0,4,48,52,7,10,2,6,42,59,"[{'event_team': 'away', 'event_time': ' 4' ', ...","[{'Player_Name': 'Yaremchuk', 'Player_Number':...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
4,Quarter-finals,03.07.2021,False,False,False,Czech Republic,Denmark,1,2,56,44,16,11,5,7,48,52,"[{'event_team': 'away', 'event_time': ' 5' ', ...","[{'Player_Name': 'Schick', 'Player_Number': '1...","[{'Player_Name': 'Dolberg', 'Player_Number': '..."


In [380]:
euro_teams = pd.DataFrame({"Teams": teams,
                           "Total goals favour": total_goals_favour , 
                           "Total goals against": total_goals_against , 
                           "Mean goals favour": , 
                           "Mean goals against": ,
                           "Accuracy": ,
                           "Win": ,
                           "Lost": ,
                           "Draw": ,
                           "Possesion mean": ,
                           "Shots on target mean": ,
                           "Shots againts mean": ,
                           "Duels win mean": ,
                           "Yellow card mean"})

SyntaxError: invalid syntax (3536828409.py, line 3)

In [334]:
#creamos una lista con los paises
teams = []
for i in df_clean["team_name_home"].unique():
    teams.append(i)
teams.sort()

In [406]:
#creamos la data para los paises
teams

['Austria',
 'Belgium',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'England',
 'Finland',
 'France',
 'Germany',
 'Hungary',
 'Italy',
 'Netherlands',
 'North Macedonia',
 'Poland',
 'Portugal',
 'Russia',
 'Scotland',
 'Slovakia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Turkey',
 'Ukraine',
 'Wales']

In [409]:
#creamos lista para total goals in favour:
total_goals_favour = []
for i in teams:
    l = df_clean[(df_clean["team_name_home"] == i)]["team_home_score"].sum()
    a = df_clean[(df_clean["team_name_away"] == i)]["team_away_score"].sum()
    total_goals_favour.append(l+a)

In [410]:
#creamos una lista para total goals against
total_goals_against = []
for i in teams:
    l = df_clean[(df_clean["team_name_home"] == i)]["team_away_score"].sum()
    a = df_clean[(df_clean["team_name_away"] == i)]["team_home_score"].sum()
    total_goals_against.append(l+a)

#goles totales a favor/ en contra
goles medios por partido a favor/ encontra
eficacia media: sumatorio (total de goles / total de disparos) / total partidos
victorias, empates, derrotas
posesion media por partido
disparos medios por partido totales
diparos medios por partido on target
disparos medios recibidos por partido
media de duelos ganados por partido
media de tarjetas amarillas / rojas
media de cambios por partido

In [349]:
df_clean["events_list"][0][0]['event_type']

'Goal'

In [359]:
events = set()
for i in df_clean["events_list"]:
    for j in i:
        events.add(j['event_type'])

In [360]:
events

{'Disallowed goal',
 'Goal',
 'Own goal',
 'PK',
 'Penalty',
 'Red card',
 'Second yellow card',
 'Substitution',
 'Yellow card'}